In [1]:
#!nvidia-smi
#using a GeForce GTX1080 Ti for reproducibility for all timing experiments

In [2]:
import torch
import torchvision
from torch import nn, optim, autograd
from torch.nn import functional as F
import numpy as np
from sklearn.metrics import roc_auc_score
import scipy
from utils.LB_utils import * 
from utils.load_not_MNIST import notMNIST
import os
import time
import matplotlib.pyplot as plt
from laplace import Laplace

s = 1
np.random.seed(s)
torch.manual_seed(s)
torch.cuda.manual_seed(s)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
cuda_status = torch.cuda.is_available()
print("device: ", device)
print("cuda status: ", cuda_status)

device:  cuda
cuda status:  True


In [4]:
### define network
class ConvNet(nn.Module):
    
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        
        self.net = torch.nn.Sequential(
            torch.nn.Conv2d(1, 16, 5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2,2),
            torch.nn.Conv2d(16, 32, 5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2,2),
            torch.nn.Flatten(),
            torch.nn.Linear(4 * 4 * 32, num_classes)
        )
    def forward(self, x):
        out = self.net(x)
        return out

In [5]:
BATCH_SIZE_TRAIN_MNIST = 128
BATCH_SIZE_TEST_MNIST = 128
MAX_ITER_MNIST = 6
LR_TRAIN_MNIST = 10e-6

In [6]:
MNIST_transform = torchvision.transforms.ToTensor()

MNIST_train = torchvision.datasets.MNIST(
        '~/data/mnist',
        train=True,
        download=True,
        transform=MNIST_transform)

MNIST_train_loader = torch.utils.data.dataloader.DataLoader(
    MNIST_train,
    batch_size=BATCH_SIZE_TRAIN_MNIST,
    shuffle=True
)


MNIST_test = torchvision.datasets.MNIST(
        '~/data/mnist',
        train=False,
        download=False,
        transform=MNIST_transform)

MNIST_test_loader = torch.utils.data.dataloader.DataLoader(
    MNIST_test,
    batch_size=BATCH_SIZE_TEST_MNIST,
    shuffle=False,
)

In [7]:
mnist_model = ConvNet().to(device)
loss_function = torch.nn.CrossEntropyLoss()

mnist_train_optimizer = torch.optim.Adam(mnist_model.parameters(), lr=1e-3, weight_decay=5e-4)
MNIST_PATH = "pretrained_weights/MNIST_pretrained_10_classes_last_layer_s{}.pth".format(s)

In [8]:
#Training routine

def train(model, train_loader, optimizer, max_iter, path, verbose=True):
    max_len = len(train_loader)

    for iter in range(max_iter):
        for batch_idx, (x, y) in enumerate(train_loader):
            
            x, y = x.to(device), y.to(device)
            
            output = model(x)

            accuracy = get_accuracy(output, y)

            loss = loss_function(output, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if verbose and batch_idx % 50 == 0:
                print(
                    "Iteration {}; {}/{} \t".format(iter, batch_idx, max_len) +
                    "Minibatch Loss %.3f  " % (loss) +
                    "Accuracy %.0f" % (accuracy * 100) + "%"
                )

    print("saving model at: {}".format(path))
    torch.save(mnist_model.state_dict(), path)

In [9]:
#train(mnist_model, MNIST_train_loader, mnist_train_optimizer, MAX_ITER_MNIST, MNIST_PATH, verbose=True)

In [10]:
#predict in distribution
MNIST_PATH = "pretrained_weights/MNIST_pretrained_10_classes_last_layer_s{}.pth".format(s)
#MNIST_PATH = "pretrained_weights/MNIST_pretrained_10_classes_last_layer.pth"

mnist_model = ConvNet().to(device)
print("loading model from: {}".format(MNIST_PATH))
mnist_model.load_state_dict(torch.load(MNIST_PATH))
mnist_model.eval()

acc = []

max_len = len(MNIST_test_loader)
for batch_idx, (x, y) in enumerate(MNIST_test_loader):

    x, y = x.to(device), y.to(device)
    output = mnist_model(x)

    accuracy = get_accuracy(output, y)
    if batch_idx % 10 == 0:
        print(
            "Batch {}/{} \t".format(batch_idx, max_len) + 
            "Accuracy %.0f" % (accuracy * 100) + "%"
        )
    acc.append(accuracy)

avg_acc = np.mean(acc)
print('overall test accuracy on MNIST: {:.02f} %'.format(avg_acc * 100))


loading model from: pretrained_weights/MNIST_pretrained_10_classes_last_layer_s1.pth
Batch 0/79 	Accuracy 100%
Batch 10/79 	Accuracy 96%
Batch 20/79 	Accuracy 98%
Batch 30/79 	Accuracy 98%
Batch 40/79 	Accuracy 100%
Batch 50/79 	Accuracy 99%
Batch 60/79 	Accuracy 100%
Batch 70/79 	Accuracy 98%
overall test accuracy on MNIST: 98.84 %


In [11]:
BATCH_SIZE_TEST_FMNIST = 128
BATCH_SIZE_TEST_KMNIST = 128

In [12]:
FMNIST_test = torchvision.datasets.FashionMNIST(
        '~/data/fmnist', train=False, download=True,
        transform=MNIST_transform)   #torchvision.transforms.ToTensor())

FMNIST_test_loader = torch.utils.data.DataLoader(
    FMNIST_test,
    batch_size=BATCH_SIZE_TEST_FMNIST, shuffle=False)

In [13]:
KMNIST_test = torchvision.datasets.KMNIST(
        '~/data/kmnist', train=False, download=True,
        transform=MNIST_transform)

KMNIST_test_loader = torch.utils.data.DataLoader(
    KMNIST_test,
    batch_size=BATCH_SIZE_TEST_KMNIST, shuffle=False)

In [14]:
#root = os.path.abspath('~/data')
root = os.path.expanduser('~/data')

# Instantiating the notMNIST dataset class we created
notMNIST_test = notMNIST(root=os.path.join(root, 'notMNIST_small'),
                               transform=MNIST_transform)

# Creating a dataloader
notMNIST_test_loader = torch.utils.data.dataloader.DataLoader(
                            dataset=notMNIST_test,
                            batch_size=BATCH_SIZE_TEST_KMNIST,
                            shuffle=False)

File F/Q3Jvc3NvdmVyIEJvbGRPYmxpcXVlLnR0Zg==.png is broken
File A/RGVtb2NyYXRpY2FCb2xkT2xkc3R5bGUgQm9sZC50dGY=.png is broken


# MAP estimate

In [15]:
targets = MNIST_test.targets.numpy()
targets_FMNIST = FMNIST_test.targets.numpy()
targets_notMNIST = notMNIST_test.targets.numpy().astype(int)
targets_KMNIST = KMNIST_test.targets.numpy()

In [16]:
MNIST_test_in_MAP = predict_MAP(mnist_model, MNIST_test_loader, device=device).cpu().numpy()
MNIST_test_out_fmnist_MAP = predict_MAP(mnist_model, FMNIST_test_loader, device=device).cpu().numpy()
MNIST_test_out_notMNIST_MAP = predict_MAP(mnist_model, notMNIST_test_loader, device=device).cpu().numpy()
MNIST_test_out_KMNIST_MAP = predict_MAP(mnist_model, KMNIST_test_loader, device=device).cpu().numpy()

In [17]:
acc_in_MAP, prob_correct_in_MAP, ent_in_MAP, MMC_in_MAP = get_in_dist_values(MNIST_test_in_MAP, targets)
acc_out_FMNIST_MAP, prob_correct_out_FMNIST_MAP, ent_out_FMNIST_MAP, MMC_out_FMNIST_MAP, auroc_out_FMNIST_MAP = get_out_dist_values(MNIST_test_in_MAP, MNIST_test_out_fmnist_MAP, targets_FMNIST)
acc_out_notMNIST_MAP, prob_correct_out_notMNIST_MAP, ent_out_notMNIST_MAP, MMC_out_notMNIST_MAP, auroc_out_notMNIST_MAP = get_out_dist_values(MNIST_test_in_MAP, MNIST_test_out_notMNIST_MAP, targets_notMNIST)
acc_out_KMNIST_MAP, prob_correct_out_KMNIST_MAP, ent_out_KMNIST_MAP, MMC_out_KMNIST_MAP, auroc_out_KMNIST_MAP = get_out_dist_values(MNIST_test_in_MAP, MNIST_test_out_KMNIST_MAP, targets_KMNIST)

In [18]:
print_in_dist_values(acc_in_MAP, prob_correct_in_MAP, ent_in_MAP, MMC_in_MAP, 'MNIST', 'MAP')
print_out_dist_values(acc_out_FMNIST_MAP, prob_correct_out_FMNIST_MAP, ent_out_FMNIST_MAP, MMC_out_FMNIST_MAP, auroc_out_FMNIST_MAP, 'MNIST', test='FMNIST', method='MAP')
print_out_dist_values(acc_out_notMNIST_MAP, prob_correct_out_notMNIST_MAP, ent_out_notMNIST_MAP, MMC_out_notMNIST_MAP, auroc_out_notMNIST_MAP, 'MNIST', test='notMNIST', method='MAP')
print_out_dist_values(acc_out_KMNIST_MAP, prob_correct_out_KMNIST_MAP, ent_out_KMNIST_MAP, MMC_out_KMNIST_MAP, auroc_out_KMNIST_MAP, 'MNIST', test='KMNIST', method='MAP')

[In, MAP, MNIST] Accuracy: 0.988; average entropy: 0.047;     MMC: 0.986; Prob @ correct: 0.100
[Out-FMNIST, MAP, MNIST] Accuracy: 0.073; Average entropy: 0.972;    MMC: 0.663; AUROC: 0.977; Prob @ correct: 0.100
[Out-notMNIST, MAP, MNIST] Accuracy: 0.148; Average entropy: 0.620;    MMC: 0.774; AUROC: 0.914; Prob @ correct: 0.100
[Out-KMNIST, MAP, MNIST] Accuracy: 0.093; Average entropy: 0.750;    MMC: 0.723; AUROC: 0.959; Prob @ correct: 0.100


In [19]:
num_samples = 100

# Diag Hessian Sampling estimate

In [20]:
la_diag = Laplace(mnist_model, 'classification', 
                     subset_of_weights='last_layer', 
                     hessian_structure='diag',
                     prior_precision=1e-0) # 5e-4 # Choose prior precision according to weight decay
la_diag.fit(MNIST_train_loader)

In [21]:
MNIST_test_in_D = predict_samples(la_diag, MNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_FMNIST_D = predict_samples(la_diag, FMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_notMNIST_D = predict_samples(la_diag, notMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_KMNIST_D = predict_samples(la_diag, KMNIST_test_loader, timing=True, device=device).cpu().numpy()

time:  1.1628369900000006
time:  0.9792774170000023
time:  0.4947420930000028
time:  0.9655261910000021


In [22]:
# compute average log-likelihood for Diag
print(torch.distributions.Categorical(torch.tensor(MNIST_test_in_D)).log_prob(torch.tensor(targets)).mean())
print(torch.distributions.Categorical(torch.tensor(MNIST_test_out_FMNIST_D)).log_prob(torch.tensor(targets_FMNIST)).mean())
print(torch.distributions.Categorical(torch.tensor(MNIST_test_out_notMNIST_D)).log_prob(torch.tensor(targets_notMNIST)).mean())
print(torch.distributions.Categorical(torch.tensor(MNIST_test_out_KMNIST_D)).log_prob(torch.tensor(targets_KMNIST)).mean())

tensor(-0.0605)
tensor(-3.9607)
tensor(-4.3349)
tensor(-4.9334)


In [23]:
import utils.scoring as scoring

In [24]:
#compute the Expected confidence estimate
print(scoring.expected_calibration_error(targets, MNIST_test_in_D))
print(scoring.expected_calibration_error(targets_FMNIST, MNIST_test_out_FMNIST_D))
print(scoring.expected_calibration_error(targets_notMNIST, MNIST_test_out_notMNIST_D))
print(scoring.expected_calibration_error(targets_KMNIST, MNIST_test_out_KMNIST_D))

0.03493014141414141
0.46846589898989915
0.409833633277876
0.43425194949494955


In [25]:
acc_in_D, prob_correct_in_D, ent_in_D, MMC_in_D = get_in_dist_values(MNIST_test_in_D, targets)
acc_out_FMNIST_D, prob_correct_out_FMNIST_D, ent_out_FMNIST_D, MMC_out_FMNIST_D, auroc_out_FMNIST_D = get_out_dist_values(MNIST_test_in_D, MNIST_test_out_FMNIST_D, targets_FMNIST)
acc_out_notMNIST_D, prob_correct_out_notMNIST_D, ent_out_notMNIST_D, MMC_out_notMNIST_D, auroc_out_notMNIST_D = get_out_dist_values(MNIST_test_in_D, MNIST_test_out_notMNIST_D, targets_notMNIST)
acc_out_KMNIST_D, prob_correct_out_KMNIST_D, ent_out_KMNIST_D, MMC_out_KMNIST_D, auroc_out_KMNIST_D = get_out_dist_values(MNIST_test_in_D, MNIST_test_out_KMNIST_D, targets_KMNIST)

In [26]:
print_in_dist_values(acc_in_D, prob_correct_in_D, ent_in_D, MMC_in_D, 'MNIST', 'Diag')
print_out_dist_values(acc_out_FMNIST_D, prob_correct_out_FMNIST_D, ent_out_FMNIST_D, MMC_out_FMNIST_D, auroc_out_FMNIST_D, 'MNIST', test='fmnist', method='Diag')
print_out_dist_values(acc_out_notMNIST_D, prob_correct_out_notMNIST_D, ent_out_notMNIST_D, MMC_out_notMNIST_D, auroc_out_notMNIST_D, 'MNIST', test='notMNIST', method='Diag')
print_out_dist_values(acc_out_KMNIST_D, prob_correct_out_KMNIST_D, ent_out_KMNIST_D, MMC_out_KMNIST_D, auroc_out_KMNIST_D, 'MNIST', test='KMNIST', method='Diag')

[In, Diag, MNIST] Accuracy: 0.989; average entropy: 0.160;     MMC: 0.956; Prob @ correct: 0.100
[Out-fmnist, Diag, MNIST] Accuracy: 0.073; Average entropy: 1.313;    MMC: 0.541; AUROC: 0.975; Prob @ correct: 0.100
[Out-notMNIST, Diag, MNIST] Accuracy: 0.149; Average entropy: 1.212;    MMC: 0.559; AUROC: 0.963; Prob @ correct: 0.100
[Out-KMNIST, Diag, MNIST] Accuracy: 0.095; Average entropy: 1.272;    MMC: 0.529; AUROC: 0.973; Prob @ correct: 0.100


# KFAC Laplace Approximation (sampling)

In [27]:
la_kron = Laplace(mnist_model, 'classification', 
                     subset_of_weights='last_layer', 
                     hessian_structure='kron',
                     prior_precision=5e-0) # 5e-4 # Choose prior precision according to weight decay
la_kron.fit(MNIST_train_loader)

/home/marius/anaconda3/lib/python3.9/site-packages/laplace/utils.py:137: UserWarning: torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.
L, _ = torch.symeig(A, upper=upper)
should be replaced with
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else 'L')
and
L, V = torch.symeig(A, eigenvectors=True)
should be replaced with
L, V = torch.linalg.eigh(A, UPLO='U' if upper else 'L') (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180549130/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:2499.)
  L, W = torch.symeig(M, eigenvectors=True)


In [28]:
MNIST_test_in_KFAC = predict_samples(la_kron, MNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_FMNIST_KFAC = predict_samples(la_kron, FMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_notMNIST_KFAC = predict_samples(la_kron, notMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_KMNIST_KFAC = predict_samples(la_kron, KMNIST_test_loader, timing=True, device=device).cpu().numpy()

time:  1.2494627059999992
time:  1.2474731460000008
time:  1.0096681889999992
time:  1.2346936679999985


In [29]:
# compute average log-likelihood for KFAC
print(torch.distributions.Categorical(torch.tensor(MNIST_test_in_KFAC)).log_prob(torch.tensor(targets)).mean())
print(torch.distributions.Categorical(torch.tensor(MNIST_test_out_FMNIST_KFAC)).log_prob(torch.tensor(targets_FMNIST)).mean())
print(torch.distributions.Categorical(torch.tensor(MNIST_test_out_notMNIST_KFAC)).log_prob(torch.tensor(targets_notMNIST)).mean())
print(torch.distributions.Categorical(torch.tensor(MNIST_test_out_KMNIST_KFAC)).log_prob(torch.tensor(targets_KMNIST)).mean())

tensor(-0.0487)
tensor(-3.7331)
tensor(-4.3672)
tensor(-4.8779)


In [30]:
# compute ECE for KFAC
print(scoring.expected_calibration_error(targets, MNIST_test_in_KFAC))
print(scoring.expected_calibration_error(targets_FMNIST, MNIST_test_out_FMNIST_KFAC))
print(scoring.expected_calibration_error(targets_notMNIST, MNIST_test_out_notMNIST_KFAC))
print(scoring.expected_calibration_error(targets_KMNIST, MNIST_test_out_KMNIST_KFAC))

0.024422959595959588
0.42929906060606066
0.40477670855100895
0.4167330707070709


In [31]:
acc_in_KFAC, prob_correct_in_KFAC, ent_in_KFAC, MMC_in_KFAC = get_in_dist_values(MNIST_test_in_KFAC, targets)
acc_out_FMNIST_KFAC, prob_correct_out_FMNIST_KFAC, ent_out_FMNIST_KFAC, MMC_out_FMNIST_KFAC, auroc_out_FMNIST_KFAC = get_out_dist_values(MNIST_test_in_KFAC, MNIST_test_out_FMNIST_KFAC, targets_FMNIST)
acc_out_notMNIST_KFAC, prob_correct_out_notMNIST_KFAC, ent_out_notMNIST_KFAC, MMC_out_notMNIST_KFAC, auroc_out_notMNIST_KFAC = get_out_dist_values(MNIST_test_in_KFAC, MNIST_test_out_notMNIST_KFAC, targets_notMNIST)
acc_out_KMNIST_KFAC, prob_correct_out_KMNIST_KFAC, ent_out_KMNIST_KFAC, MMC_out_KMNIST_KFAC, auroc_out_KMNIST_KFAC = get_out_dist_values(MNIST_test_in_KFAC, MNIST_test_out_KMNIST_KFAC, targets_KMNIST)

In [32]:
print_in_dist_values(acc_in_KFAC, prob_correct_in_KFAC, ent_in_KFAC, MMC_in_KFAC, 'MNIST', 'KFAC')
print_out_dist_values(acc_out_FMNIST_KFAC, prob_correct_out_FMNIST_KFAC, ent_out_FMNIST_KFAC, MMC_out_FMNIST_KFAC, auroc_out_FMNIST_KFAC, 'MNIST', test='fmnist', method='KFAC')
print_out_dist_values(acc_out_notMNIST_KFAC, prob_correct_out_notMNIST_KFAC, ent_out_notMNIST_KFAC, MMC_out_notMNIST_KFAC, auroc_out_notMNIST_KFAC, 'MNIST', test='notMNIST', method='KFAC')
print_out_dist_values(acc_out_KMNIST_KFAC, prob_correct_out_KMNIST_KFAC, ent_out_KMNIST_KFAC, MMC_out_KMNIST_KFAC, auroc_out_KMNIST_KFAC, 'MNIST', test='KMNIST', method='KFAC')

[In, KFAC, MNIST] Accuracy: 0.988; average entropy: 0.111;     MMC: 0.967; Prob @ correct: 0.100
[Out-fmnist, KFAC, MNIST] Accuracy: 0.072; Average entropy: 1.402;    MMC: 0.501; AUROC: 0.985; Prob @ correct: 0.100
[Out-notMNIST, KFAC, MNIST] Accuracy: 0.148; Average entropy: 1.228;    MMC: 0.551; AUROC: 0.969; Prob @ correct: 0.100
[Out-KMNIST, KFAC, MNIST] Accuracy: 0.094; Average entropy: 1.315;    MMC: 0.511; AUROC: 0.982; Prob @ correct: 0.100


# Laplace Bridge (Diagonal)

In [33]:
MNIST_test_in_LB_D = predict_LB(la_diag, MNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_FMNIST_LB_D = predict_LB(la_diag, FMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_notMNIST_LB_D = predict_LB(la_diag, notMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_KMNIST_LB_D = predict_LB(la_diag, KMNIST_test_loader, timing=True, device=device).cpu().numpy()

step0, f_mu[0]:  tensor([ -3.6094,  -2.8937,   1.1239,   1.7301,  -9.0167,  -3.9072, -17.8319,
         13.2881,  -5.2003,   0.8784], device='cuda:0')
step0, f_var[0]:  tensor([[ 9.1984,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000, 19.1064,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  3.7570,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  3.2656,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  8.3827,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  6.0570,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 21.9312,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.00

step0, f_mu[0]:  tensor([ -7.5057, -11.0894,  -6.9948,  -1.1693,   1.8382,  -0.7919, -13.6171,
          1.7905,  -1.8001,  10.2949], device='cuda:0')
step0, f_var[0]:  tensor([[ 8.6273,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000, 22.3177,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  4.7154,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  3.1492,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  3.1418,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.8023,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 10.9577,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.00

step0, f_mu[0]:  tensor([ -2.9080, -11.5330,  -6.2933,  -1.2008,   0.4824,  -4.2224, -13.5404,
         -0.4246,  -4.2598,  11.8633], device='cuda:0')
step0, f_var[0]:  tensor([[10.3306,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000, 43.7253,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  6.6535,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  5.5132,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  4.3741,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  6.5211,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 12.8647,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.00

step0, f_mu[0]:  tensor([ -6.8779,  -3.1840,   6.9303,   2.6190,  -6.5269, -11.0182, -20.9386,
         15.0805,  -5.3011,  -5.8907], device='cuda:0')
step0, f_var[0]:  tensor([[13.9824,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000, 24.1308,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  3.7583,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  4.9627,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  9.5200,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 11.0143,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 24.7522,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.00

step0, f_mu[0]:  tensor([ 13.0570, -15.2212,  -5.3315,  -9.7353, -11.3007,   1.5989,   2.8546,
         -5.1484,  -3.9605,  -2.5558], device='cuda:0')
step0, f_var[0]:  tensor([[ 9.2979,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000, 90.9389,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000, 11.0750,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000, 10.4740,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000, 29.4905,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  8.2674,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 13.9678,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.00

...

step0, f_mu[0]:  tensor([ 2.2914, -0.5672, -0.1310, -7.0403,  0.4212, -0.8954,  1.7621, -1.5900,
         0.1058, -4.6421], device='cuda:0')
step0, f_var[0]:  tensor([[ 2.4297,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000, 15.8350,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  2.1857,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  2.2434,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  2.5646,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.9150,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.2008,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  2.616

step0, f_mu[0]:  tensor([-3.3540, -4.5152, -2.5444, -3.8279,  3.1069, -4.1957, -5.1523, -0.0104,
        -0.5599, -3.0251], device='cuda:0')
step0, f_var[0]:  tensor([[ 7.7231,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000, 12.1979,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  3.5464,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  3.6092,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  2.5903,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  5.0948,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  6.4599,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.343

step0, f_mu[0]:  tensor([  4.6695, -12.3448,   5.8070,   0.7010, -14.7909,  -7.0551,  -1.2691,
         -4.9956,   4.4095,  -3.3876], device='cuda:0')
step0, f_var[0]:  tensor([[ 6.9713,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000, 27.4533,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  3.9389,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  4.4555,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  8.2284,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  5.3663,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 11.4618,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.00

step0, f_mu[0]:  tensor([-2.1541, -8.2685, -4.1841,  2.8451, -3.9501, -1.8986, -0.4488, -1.9184,
        -0.3712,  1.2918], device='cuda:0')
step0, f_var[0]:  tensor([[ 4.8074,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000, 21.2241,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  2.7811,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  2.5758,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  3.2657,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  2.8287,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  4.6244,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.183

step0, f_mu[0]:  tensor([-1.5137, -6.8763, -2.8725, -0.3809, -1.8610, -2.9026, -3.5073,  0.0340,
         0.7767, -0.3569], device='cuda:0')
step0, f_var[0]:  tensor([[ 4.7254,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000, 10.8481,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  2.4781,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  2.3369,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  2.5971,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  2.7414,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  5.1281,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  2.472

In [34]:
# # compute average log-likelihood for LB
print(torch.distributions.Categorical(torch.tensor(MNIST_test_in_LB_D)).log_prob(torch.tensor(targets)).mean())
print(torch.distributions.Categorical(torch.tensor(MNIST_test_out_FMNIST_LB_D)).log_prob(torch.tensor(targets_FMNIST)).mean())
print(torch.distributions.Categorical(torch.tensor(MNIST_test_out_notMNIST_LB_D)).log_prob(torch.tensor(targets_notMNIST)).mean())
print(torch.distributions.Categorical(torch.tensor(MNIST_test_out_KMNIST_LB_D)).log_prob(torch.tensor(targets_KMNIST)).mean())

tensor(-0.0409)
tensor(-3.6545)
tensor(-5.5852)
tensor(-5.5203)


In [35]:
#compute ECE for LB
print(scoring.expected_calibration_error(targets, MNIST_test_in_LB_D))
print(scoring.expected_calibration_error(targets_FMNIST, MNIST_test_out_FMNIST_LB_D))
print(scoring.expected_calibration_error(targets_notMNIST, MNIST_test_out_notMNIST_LB_D))
print(scoring.expected_calibration_error(targets_KMNIST, MNIST_test_out_KMNIST_LB_D))

0.011439090909090939
0.48319700000000004
0.6082117478998488
0.5931453232323233


In [36]:
acc_in_LB_D, prob_correct_in_LB_D, ent_in_LB_D, MMC_in_LB_D = get_in_dist_values(MNIST_test_in_LB_D, targets)
acc_out_FMNIST_LB_D, prob_correct_out_FMNIST_LB_D, ent_out_FMNIST_LB_D, MMC_out_FMNIST_LB_D, auroc_out_FMNIST_LB_D = get_out_dist_values(MNIST_test_in_LB_D, MNIST_test_out_FMNIST_LB_D, targets_FMNIST)
acc_out_notMNIST_LB_D, prob_correct_out_notMNIST_LB_D, ent_out_notMNIST_LB_D, MMC_out_notMNIST_LB_D, auroc_out_notMNIST_LB_D = get_out_dist_values(MNIST_test_in_LB_D, MNIST_test_out_notMNIST_LB_D, targets_notMNIST)
acc_out_KMNIST_LB_D, prob_correct_out_KMNIST_LB_D, ent_out_KMNIST_LB_D, MMC_out_KMNIST_LB_D, auroc_out_KMNIST_LB_D = get_out_dist_values(MNIST_test_in_LB_D, MNIST_test_out_KMNIST_LB_D, targets_KMNIST)

In [37]:
print_in_dist_values(acc_in_LB_D, prob_correct_in_LB_D, ent_in_LB_D, MMC_in_LB_D, 'MNIST', 'LB_D')
print_out_dist_values(acc_out_FMNIST_LB_D, prob_correct_out_FMNIST_LB_D, ent_out_FMNIST_LB_D, MMC_out_FMNIST_LB_D, auroc_out_FMNIST_LB_D, 'MNIST', test='fmnist', method='LB_D')
print_out_dist_values(acc_out_notMNIST_LB_D, prob_correct_out_notMNIST_LB_D, ent_out_notMNIST_LB_D, MMC_out_notMNIST_LB_D, auroc_out_notMNIST_LB_D, 'MNIST', test='notMNIST', method='LB_D')
print_out_dist_values(acc_out_KMNIST_LB_D, prob_correct_out_KMNIST_LB_D, ent_out_KMNIST_LB_D, MMC_out_KMNIST_LB_D, auroc_out_KMNIST_LB_D, 'MNIST', test='KMNIST', method='LB_D')

[In, LB_D, MNIST] Accuracy: 0.987; average entropy: 0.061;     MMC: 0.982; Prob @ correct: 0.100
[Out-fmnist, LB_D, MNIST] Accuracy: 0.082; Average entropy: 1.324;    MMC: 0.565; AUROC: 0.982; Prob @ correct: 0.100
[Out-notMNIST, LB_D, MNIST] Accuracy: 0.152; Average entropy: 0.737;    MMC: 0.735; AUROC: 0.931; Prob @ correct: 0.100
[Out-KMNIST, LB_D, MNIST] Accuracy: 0.099; Average entropy: 0.866;    MMC: 0.692; AUROC: 0.960; Prob @ correct: 0.100


# KFAC Laplace Bridge

In [38]:
MNIST_test_in_LB_KFAC = predict_LB(la_kron, MNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_FMNIST_LB_KFAC = predict_LB(la_kron, FMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_notMNIST_LB_KFAC = predict_LB(la_kron, notMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_KMNIST_LB_KFAC = predict_LB(la_kron, KMNIST_test_loader, timing=True, device=device).cpu().numpy()

step0, f_mu[0]:  tensor([ -3.6094,  -2.8937,   1.1239,   1.7301,  -9.0167,  -3.9072, -17.8319,
         13.2881,  -5.2003,   0.8784], device='cuda:0')
step0, f_var[0]:  tensor([[42.6037, 40.4586, 40.5774, 40.5005, 40.5371, 40.5741, 40.8008, 40.5139,
         40.6334, 40.5754],
        [40.4586, 42.3545, 40.7232, 40.5542, 40.6854, 40.5083, 40.5141, 40.6926,
         40.6774, 40.6066],
        [40.5774, 40.7232, 41.9556, 40.7366, 40.6178, 40.5562, 40.4891, 40.7880,
         40.7114, 40.6196],
        [40.5005, 40.5542, 40.7366, 41.9227, 40.5880, 40.8414, 40.4990, 40.7278,
         40.7166, 40.6885],
        [40.5371, 40.6854, 40.6178, 40.5880, 41.8064, 40.5738, 40.6068, 40.7205,
         40.6691, 40.9704],
        [40.5741, 40.5083, 40.5562, 40.8414, 40.5738, 42.0326, 40.7329, 40.5634,
         40.7450, 40.6472],
        [40.8008, 40.5141, 40.4891, 40.4990, 40.6068, 40.7329, 42.4786, 40.4548,
         40.6701, 40.5288],
        [40.5139, 40.6926, 40.7880, 40.7278, 40.7205, 40.5634, 40.45

step0, f_mu[0]:  tensor([ 8.4595, -6.3176, -0.2372, -6.8238, -0.2402, -8.3703,  2.6686, -7.6730,
        -4.2665, -0.5893], device='cuda:0')
step0, f_var[0]:  tensor([[54.3705, 48.9291, 49.2248, 49.0313, 49.1222, 49.2127, 49.7731, 49.0660,
         49.3633, 49.2192],
        [48.9291, 53.7535, 49.5854, 49.1618, 49.4913, 49.0497, 49.0675, 49.5080,
         49.4736, 49.2922],
        [49.2248, 49.5854, 52.7533, 49.6214, 49.3213, 49.1674, 49.0035, 49.7507,
         49.5599, 49.3246],
        [49.0313, 49.1618, 49.6214, 52.6686, 49.2457, 49.8865, 49.0263, 49.5987,
         49.5720, 49.5003],
        [49.1222, 49.4913, 49.3213, 49.2457, 52.3732, 49.2115, 49.2980, 49.5795,
         49.4520, 50.2181],
        [49.2127, 49.0497, 49.1674, 49.8865, 49.2115, 52.9466, 49.6105, 49.1856,
         49.6445, 49.3974],
        [49.7732, 49.0675, 49.0035, 49.0263, 49.2980, 49.6105, 54.0596, 48.9178,
         49.4554, 49.1005],
        [49.0660, 49.5080, 49.7507, 49.5987, 49.5795, 49.1856, 48.9178, 52.490

step0, f_var[0]:  tensor([[27.2891, 24.1199, 24.2923, 24.1794, 24.2324, 24.2852, 24.6119, 24.1997,
         24.3731, 24.2890],
        [24.1199, 26.9294, 24.5026, 24.2555, 24.4477, 24.1902, 24.2006, 24.4574,
         24.4374, 24.3315],
        [24.2924, 24.5026, 26.3461, 24.5236, 24.3485, 24.2588, 24.1633, 24.5990,
         24.4877, 24.3505],
        [24.1794, 24.2555, 24.5236, 26.2966, 24.3044, 24.6782, 24.1765, 24.5103,
         24.4948, 24.4529],
        [24.2324, 24.4477, 24.3485, 24.3044, 26.1244, 24.2845, 24.3350, 24.4991,
         24.4248, 24.8716],
        [24.2852, 24.1902, 24.2588, 24.6782, 24.2845, 26.4588, 24.5172, 24.2694,
         24.5371, 24.3929],
        [24.6119, 24.2006, 24.1633, 24.1765, 24.3350, 24.5172, 27.1078, 24.1133,
         24.4268, 24.2198],
        [24.1997, 24.4574, 24.5990, 24.5103, 24.4991, 24.2694, 24.1133, 26.1926,
         24.3841, 24.6475],
        [24.3731, 24.4374, 24.4877, 24.4948, 24.4248, 24.5371, 24.4268, 24.3841,
         25.8048, 24.5017],
 

step0, f_mu[0]:  tensor([-1.7800,  7.8760, -1.3957, -7.2499,  0.1324, -3.8599, -0.0693, -1.4312,
        -0.0469, -5.0897], device='cuda:0')
step0, f_var[0]:  tensor([[21.5345, 20.6934, 20.7418, 20.7108, 20.7258, 20.7412, 20.8343, 20.7160,
         20.7648, 20.7410],
        [20.6934, 21.4323, 20.8017, 20.7333, 20.7862, 20.7142, 20.7159, 20.7894,
         20.7825, 20.7548],
        [20.7418, 20.8017, 21.2712, 20.8065, 20.7591, 20.7341, 20.7061, 20.8272,
         20.7959, 20.7601],
        [20.7108, 20.7333, 20.8065, 21.2584, 20.7473, 20.8484, 20.7105, 20.8031,
         20.7983, 20.7872],
        [20.7259, 20.7862, 20.7591, 20.7473, 21.2125, 20.7412, 20.7537, 20.8005,
         20.7794, 20.8982],
        [20.7412, 20.7142, 20.7342, 20.8484, 20.7412, 21.3022, 20.8055, 20.7370,
         20.8095, 20.7704],
        [20.8343, 20.7159, 20.7061, 20.7105, 20.7537, 20.8055, 21.4833, 20.6921,
         20.7795, 20.7227],
        [20.7160, 20.7894, 20.8272, 20.8031, 20.8005, 20.7370, 20.6921, 21.230

step0, f_var[0]:  tensor([[61.9231, 57.7097, 57.9391, 57.7889, 57.8594, 57.9296, 58.3642, 57.8159,
         58.0465, 57.9347],
        [57.7097, 61.4446, 58.2187, 57.8901, 58.1457, 57.8032, 57.8170, 58.1586,
         58.1320, 57.9913],
        [57.9391, 58.2187, 60.6688, 58.2466, 58.0139, 57.8945, 57.7674, 58.3469,
         58.1989, 58.0165],
        [57.7889, 57.8901, 58.2466, 60.6031, 57.9552, 58.4523, 57.7851, 58.2290,
         58.2084, 58.1527],
        [57.8594, 58.1457, 58.0139, 57.9553, 60.3742, 57.9287, 57.9958, 58.2142,
         58.1153, 58.7093],
        [57.9296, 57.8032, 57.8945, 58.4523, 57.9287, 60.8188, 58.2381, 57.9087,
         58.2645, 58.0729],
        [58.3642, 57.8170, 57.7674, 57.7851, 57.9958, 58.2381, 61.6819, 57.7009,
         58.1179, 57.8426],
        [57.8159, 58.1586, 58.3469, 58.2290, 58.2142, 57.9087, 57.7009, 60.4649,
         58.0612, 58.4114],
        [58.0464, 58.1320, 58.1989, 58.2084, 58.1153, 58.2645, 58.1179, 58.0612,
         59.9492, 58.2175],
 

...

step0, f_mu[0]:  tensor([-3.3718, -4.2952,  4.6727, -0.3328, -1.6820, -7.8870, -4.4332, -2.2254,
         0.3621, -5.0100], device='cuda:0')
step0, f_var[0]:  tensor([[34.7160, 24.4937, 25.0190, 24.6669, 24.8269, 24.9820, 25.9613, 24.7345,
         25.2616, 25.0072],
        [24.4937, 33.6340, 25.6496, 24.8861, 25.4835, 24.6957, 24.7407, 25.5080,
         25.4614, 25.1164],
        [25.0190, 25.6496, 31.8270, 25.7256, 25.1735, 24.8969, 24.6200, 25.9612,
         25.6226, 25.1737],
        [24.6669, 24.8861, 25.7256, 31.6638, 25.0326, 26.2140, 24.6529, 25.6823,
         25.6401, 25.5049],
        [24.8269, 25.4835, 25.1735, 25.0326, 31.1115, 24.9774, 25.1497, 25.6413,
         25.4180, 26.8549],
        [24.9820, 24.6957, 24.8969, 26.2140, 24.9774, 32.1767, 25.6976, 24.9308,
         25.7767, 25.3215],
        [25.9613, 24.7407, 24.6200, 24.6529, 25.1497, 25.6976, 34.1665, 24.4670,
         25.4288, 24.7844],
        [24.7345, 25.5080, 25.9612, 25.6823, 25.6413, 24.9308, 24.4670, 31.332

step0, f_mu[0]:  tensor([ -2.8484,   3.3361,  -0.6029, -10.5454,   4.8237,  -1.3438,  -0.5975,
         -0.3477,  -1.2627,  -6.8201], device='cuda:0')
step0, f_var[0]:  tensor([[34.2250, 21.7356, 22.3676, 21.9429, 22.1350, 22.3213, 23.4978, 22.0249,
         22.6595, 22.3537],
        [21.7356, 32.9248, 23.1253, 22.2051, 22.9259, 21.9773, 22.0328, 22.9545,
         22.9009, 22.4812],
        [22.3676, 23.1253, 30.7456, 23.2189, 22.5513, 22.2175, 21.8869, 23.5037,
         23.0967, 22.5498],
        [21.9429, 22.2051, 23.2189, 30.5465, 22.3801, 23.8093, 21.9252, 23.1664,
         23.1165, 22.9526],
        [22.1350, 22.9259, 22.5513, 22.3801, 29.8748, 22.3147, 22.5256, 23.1148,
         22.8472, 24.5939],
        [22.3213, 21.9773, 22.2175, 23.8093, 22.3147, 31.1669, 23.1836, 22.2586,
         23.2824, 22.7317],
        [23.4978, 22.0328, 21.8869, 21.9252, 22.5256, 23.1836, 33.5642, 21.7028,
         22.8615, 22.0829],
        [22.0249, 22.9545, 23.5037, 23.1664, 23.1148, 22.2586, 21.70

step0, f_mu[0]:  tensor([-4.4973, -6.2037, -3.3750,  4.2487, -5.1242,  0.5486, -8.6510, -0.8557,
        -2.2925,  0.8212], device='cuda:0')
step0, f_var[0]:  tensor([[32.7729, 25.4034, 25.7790, 25.5271, 25.6414, 25.7524, 26.4524, 25.5755,
         25.9526, 25.7708],
        [25.4034, 31.9994, 26.2298, 25.6837, 26.1112, 25.5477, 25.5800, 26.1286,
         26.0958, 25.8479],
        [25.7790, 26.2298, 30.7064, 26.2847, 25.8892, 25.6911, 25.4937, 26.4535,
         26.2116, 25.8888],
        [25.5271, 25.6837, 26.2847, 30.5890, 25.7879, 26.6344, 25.5169, 26.2537,
         26.2236, 26.1267],
        [25.6414, 26.1112, 25.8892, 25.7879, 30.1919, 25.7487, 25.8728, 26.2236,
         26.0643, 27.0968],
        [25.7524, 25.5477, 25.6911, 26.6344, 25.7487, 30.9564, 26.2642, 25.7154,
         26.3217, 25.9956],
        [26.4524, 25.5800, 25.4937, 25.5169, 25.8728, 26.2642, 32.3801, 25.3843,
         26.0724, 25.6107],
        [25.5755, 26.1286, 26.4535, 26.2536, 26.2236, 25.7154, 25.3843, 30.351

step0, f_mu[0]:  tensor([ -5.7261,  -0.1351,   6.5100,   0.2276,  -3.6600, -12.8357,  -5.8143,
         -0.4559,   2.9008, -10.1335], device='cuda:0')
step0, f_var[0]:  tensor([[46.8736, 34.9512, 35.5618, 35.1539, 35.3398, 35.5215, 36.6626, 35.2314,
         35.8448, 35.5489],
        [34.9512, 45.6135, 36.2965, 35.4108, 36.1037, 35.1882, 35.2380, 36.1328,
         36.0769, 35.6779],
        [35.5618, 36.2965, 43.5171, 36.3835, 35.7441, 35.4225, 35.0991, 36.6570,
         36.2638, 35.7441],
        [35.1539, 35.4108, 36.3835, 43.3283, 35.5805, 36.9489, 35.1382, 36.3337,
         36.2840, 36.1279],
        [35.3398, 36.1037, 35.7441, 35.5805, 42.6870, 35.5158, 35.7142, 36.2857,
         36.0264, 37.6926],
        [35.5215, 35.1882, 35.4225, 36.9489, 35.5158, 43.9222, 36.3518, 35.4617,
         36.4421, 35.9147],
        [36.6626, 35.2380, 35.0991, 35.1382, 35.7142, 36.3518, 46.2344, 34.9215,
         36.0389, 35.2906],
        [35.2314, 36.1328, 36.6570, 36.3337, 36.2857, 35.4617, 34.92

step0, f_mu[0]:  tensor([-7.4551, -1.8317, -5.6707, -4.0425,  3.0625,  3.0362, -6.1932, -4.9783,
        -2.5884, -2.1859], device='cuda:0')
step0, f_var[0]:  tensor([[47.2520, 31.7689, 32.5486, 32.0234, 32.2601, 32.4891, 33.9378, 32.1254,
         32.9082, 32.5312],
        [31.7689, 45.6502, 33.4817, 32.3448, 33.2359, 32.0651, 32.1357, 33.2704,
         33.2065, 32.6853],
        [32.5486, 33.4817, 42.9573, 33.5992, 32.7726, 32.3603, 31.9547, 33.9513,
         33.4493, 32.7699],
        [32.0234, 32.3448, 33.5991, 42.7096, 32.5604, 34.3302, 32.0007, 33.5339,
         33.4731, 33.2696],
        [32.2601, 33.2359, 32.7726, 32.5604, 41.8768, 32.4805, 32.7436, 33.4692,
         33.1396, 35.3062],
        [32.4891, 32.0651, 32.3603, 34.3302, 32.4805, 43.4779, 33.5541, 32.4112,
         33.6790, 32.9972],
        [33.9378, 32.1357, 31.9547, 32.0007, 32.7436, 33.5542, 46.4372, 31.7275,
         33.1579, 32.1953],
        [32.1254, 33.2704, 33.9513, 33.5339, 33.4692, 32.4112, 31.7275, 42.212

In [39]:
# compute average log-likelihood for LB KFAC
print(torch.distributions.Categorical(torch.tensor(MNIST_test_in_LB_KFAC)).log_prob(torch.tensor(targets)).mean())
print(torch.distributions.Categorical(torch.tensor(MNIST_test_out_FMNIST_LB_KFAC)).log_prob(torch.tensor(targets_FMNIST)).mean())
print(torch.distributions.Categorical(torch.tensor(MNIST_test_out_notMNIST_LB_KFAC)).log_prob(torch.tensor(targets_notMNIST)).mean())
print(torch.distributions.Categorical(torch.tensor(MNIST_test_out_KMNIST_LB_KFAC)).log_prob(torch.tensor(targets_KMNIST)).mean())

tensor(-0.0342)
tensor(-3.8378)
tensor(-6.0142)
tensor(-6.0235)


In [40]:
print(scoring.expected_calibration_error(targets, MNIST_test_in_LB_KFAC))
print(scoring.expected_calibration_error(targets_FMNIST, MNIST_test_out_FMNIST_LB_KFAC))
print(scoring.expected_calibration_error(targets_notMNIST, MNIST_test_out_notMNIST_LB_KFAC))
print(scoring.expected_calibration_error(targets_KMNIST, MNIST_test_out_KMNIST_LB_KFAC))

0.011948898989899039
0.5424864242424243
0.6367586083004797
0.6168500707070707


In [41]:
acc_in_LB_KFAC, prob_correct_in_LB_KFAC, ent_in_LB_KFAC, MMC_in_LB_KFAC = get_in_dist_values(MNIST_test_in_LB_KFAC, targets)
acc_out_FMNIST_LB_KFAC, prob_correct_out_FMNIST_LB_KFAC, ent_out_FMNIST_LB_KFAC, MMC_out_FMNIST_LB_KFAC, auroc_out_FMNIST_LB_KFAC = get_out_dist_values(MNIST_test_in_LB_KFAC, MNIST_test_out_FMNIST_LB_KFAC, targets_FMNIST)
acc_out_notMNIST_LB_KFAC, prob_correct_out_notMNIST_LB_KFAC, ent_out_notMNIST_LB_KFAC, MMC_out_notMNIST_LB_KFAC, auroc_out_notMNIST_LB_KFAC = get_out_dist_values(MNIST_test_in_LB_KFAC, MNIST_test_out_notMNIST_LB_KFAC, targets_notMNIST)
acc_out_KMNIST_LB_KFAC, prob_correct_out_KMNIST_LB_KFAC, ent_out_KMNIST_LB_KFAC, MMC_out_KMNIST_LB_KFAC, auroc_out_KMNIST_LB_KFAC = get_out_dist_values(MNIST_test_in_LB_KFAC, MNIST_test_out_KMNIST_LB_KFAC, targets_KMNIST)

In [42]:
print_in_dist_values(acc_in_LB_KFAC, prob_correct_in_LB_KFAC, ent_in_LB_KFAC, MMC_in_LB_KFAC, 'MNIST', 'LB_KFAC')
print_out_dist_values(acc_out_FMNIST_LB_KFAC, prob_correct_out_FMNIST_LB_KFAC, ent_out_FMNIST_LB_KFAC, MMC_out_FMNIST_LB_KFAC, auroc_out_FMNIST_LB_KFAC, 'MNIST', test='fmnist', method='LB_KFAC')
print_out_dist_values(acc_out_notMNIST_LB_KFAC, prob_correct_out_notMNIST_LB_KFAC, ent_out_notMNIST_LB_KFAC, MMC_out_notMNIST_LB_KFAC, auroc_out_notMNIST_LB_KFAC, 'MNIST', test='notMNIST', method='LB_KFAC')
print_out_dist_values(acc_out_KMNIST_LB_KFAC, prob_correct_out_KMNIST_LB_KFAC, ent_out_KMNIST_LB_KFAC, MMC_out_KMNIST_LB_KFAC, auroc_out_KMNIST_LB_KFAC, 'MNIST', test='KMNIST', method='LB_KFAC')

[In, LB_KFAC, MNIST] Accuracy: 0.988; average entropy: 0.051;     MMC: 0.985; Prob @ correct: 0.100
[Out-fmnist, LB_KFAC, MNIST] Accuracy: 0.075; Average entropy: 1.202;    MMC: 0.617; AUROC: 0.978; Prob @ correct: 0.100
[Out-notMNIST, LB_KFAC, MNIST] Accuracy: 0.147; Average entropy: 0.685;    MMC: 0.757; AUROC: 0.912; Prob @ correct: 0.100
[Out-KMNIST, LB_KFAC, MNIST] Accuracy: 0.094; Average entropy: 0.806;    MMC: 0.711; AUROC: 0.957; Prob @ correct: 0.100


In [43]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

# Compare to extended MacKay approach

In [ ]:
MNIST_test_in_EMK_D = predict_extended_MacKay(la_diag, MNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_FMNIST_EMK_D = predict_extended_MacKay(la_diag, FMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_notMNIST_EMK_D = predict_extended_MacKay(la_diag, notMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_KMNIST_EMK_D = predict_extended_MacKay(la_diag, KMNIST_test_loader, timing=True, device=device).cpu().numpy()

In [ ]:
acc_in_EMK, prob_correct_in_EMK, ent_in_EMK, MMC_in_EMK = get_in_dist_values(MNIST_test_in_EMK_D, targets)
acc_out_FMNIST_EMK, prob_correct_out_FMNIST_EMK, ent_out_FMNIST_EMK, MMC_out_FMNIST_EMK, auroc_out_FMNIST_EMK = get_out_dist_values(MNIST_test_in_EMK_D, MNIST_test_out_FMNIST_EMK_D, targets_FMNIST)
acc_out_notMNIST_EMK, prob_correct_out_notMNIST_EMK, ent_out_notMNIST_EMK, MMC_out_notMNIST_EMK, auroc_out_notMNIST_EMK = get_out_dist_values(MNIST_test_in_EMK_D, MNIST_test_out_notMNIST_EMK_D, targets_notMNIST)
acc_out_KMNIST_EMK, prob_correct_out_KMNIST_EMK, ent_out_KMNIST_EMK, MMC_out_KMNIST_EMK, auroc_out_KMNIST_EMK = get_out_dist_values(MNIST_test_in_EMK_D, MNIST_test_out_KMNIST_EMK_D, targets_KMNIST)

In [ ]:
print_in_dist_values(acc_in_EMK, prob_correct_in_EMK, ent_in_EMK, MMC_in_EMK, 'MNIST', 'EMK')
print_out_dist_values(acc_out_FMNIST_EMK, prob_correct_out_FMNIST_EMK, ent_out_FMNIST_EMK, MMC_out_FMNIST_EMK, auroc_out_FMNIST_EMK, 'MNIST', test='fmnist', method='EMK')
print_out_dist_values(acc_out_notMNIST_EMK, prob_correct_out_notMNIST_EMK, ent_out_notMNIST_EMK, MMC_out_notMNIST_EMK, auroc_out_notMNIST_EMK, 'MNIST', test='notMNIST', method='EMK')
print_out_dist_values(acc_out_KMNIST_EMK, prob_correct_out_KMNIST_EMK, ent_out_KMNIST_EMK, MMC_out_KMNIST_EMK, auroc_out_KMNIST_EMK, 'MNIST', test='KMNIST', method='EMK')

### EMK KFAC

In [ ]:
MNIST_test_in_EMK_K = predict_extended_MacKay(la_kron, MNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_FMNIST_EMK_K = predict_extended_MacKay(la_kron, FMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_notMNIST_EMK_K = predict_extended_MacKay(la_kron, notMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_KMNIST_EMK_K = predict_extended_MacKay(la_kron, KMNIST_test_loader, timing=True, device=device).cpu().numpy()

In [ ]:
acc_in_EMK, prob_correct_in_EMK, ent_in_EMK, MMC_in_EMK = get_in_dist_values(MNIST_test_in_EMK_K, targets)
acc_out_FMNIST_EMK, prob_correct_out_FMNIST_EMK, ent_out_FMNIST_EMK, MMC_out_FMNIST_EMK, auroc_out_FMNIST_EMK = get_out_dist_values(MNIST_test_in_EMK_K, MNIST_test_out_FMNIST_EMK_K, targets_FMNIST)
acc_out_notMNIST_EMK, prob_correct_out_notMNIST_EMK, ent_out_notMNIST_EMK, MMC_out_notMNIST_EMK, auroc_out_notMNIST_EMK = get_out_dist_values(MNIST_test_in_EMK_K, MNIST_test_out_notMNIST_EMK_K, targets_notMNIST)
acc_out_KMNIST_EMK, prob_correct_out_KMNIST_EMK, ent_out_KMNIST_EMK, MMC_out_KMNIST_EMK, auroc_out_KMNIST_EMK = get_out_dist_values(MNIST_test_in_EMK_K, MNIST_test_out_KMNIST_EMK_K, targets_KMNIST)

In [ ]:
print_in_dist_values(acc_in_EMK, prob_correct_in_EMK, ent_in_EMK, MMC_in_EMK, 'MNIST', 'EMK')
print_out_dist_values(acc_out_FMNIST_EMK, prob_correct_out_FMNIST_EMK, ent_out_FMNIST_EMK, MMC_out_FMNIST_EMK, auroc_out_FMNIST_EMK, 'MNIST', test='fmnist', method='EMK')
print_out_dist_values(acc_out_notMNIST_EMK, prob_correct_out_notMNIST_EMK, ent_out_notMNIST_EMK, MMC_out_notMNIST_EMK, auroc_out_notMNIST_EMK, 'MNIST', test='notMNIST', method='EMK')
print_out_dist_values(acc_out_KMNIST_EMK, prob_correct_out_KMNIST_EMK, ent_out_KMNIST_EMK, MMC_out_KMNIST_EMK, auroc_out_KMNIST_EMK, 'MNIST', test='KMNIST', method='EMK')

# Compare to Second-order Delta Posterior Predictive

In [ ]:
MNIST_test_in_SODPP_D = predict_second_order_dpp(la_diag, MNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_FMNIST_SODPP_D = predict_second_order_dpp(la_diag, FMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_notMNIST_SODPP_D = predict_second_order_dpp(la_diag, notMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_KMNIST_SODPP_D = predict_second_order_dpp(la_diag, KMNIST_test_loader, timing=True, device=device).cpu().numpy()

In [ ]:
acc_in_SODPP, prob_correct_in_SODPP, ent_in_SODPP, MMC_in_SODPP = get_in_dist_values(MNIST_test_in_SODPP_D, targets)
acc_out_FMNIST_SODPP, prob_correct_out_FMNIST_SODPP, ent_out_FMNIST_SODPP, MMC_out_FMNIST_SODPP, auroc_out_FMNIST_SODPP = get_out_dist_values(MNIST_test_in_SODPP_D, MNIST_test_out_FMNIST_SODPP_D, targets_FMNIST)
acc_out_notMNIST_SODPP, prob_correct_out_notMNIST_SODPP, ent_out_notMNIST_SODPP, MMC_out_notMNIST_SODPP, auroc_out_notMNIST_SODPP = get_out_dist_values(MNIST_test_in_SODPP_D, MNIST_test_out_notMNIST_SODPP_D, targets_notMNIST)
acc_out_KMNIST_SODPP, prob_correct_out_KMNIST_SODPP, ent_out_KMNIST_SODPP, MMC_out_KMNIST_SODPP, auroc_out_KMNIST_SODPP = get_out_dist_values(MNIST_test_in_SODPP_D, MNIST_test_out_KMNIST_SODPP_D, targets_KMNIST)

In [ ]:
print_in_dist_values(acc_in_SODPP, prob_correct_in_SODPP, ent_in_SODPP, MMC_in_SODPP, 'MNIST', 'SODPP')
print_out_dist_values(acc_out_FMNIST_SODPP, prob_correct_out_FMNIST_SODPP, ent_out_FMNIST_SODPP, MMC_out_FMNIST_SODPP, auroc_out_FMNIST_SODPP, 'MNIST', test='fmnist', method='SODPP')
print_out_dist_values(acc_out_notMNIST_SODPP, prob_correct_out_notMNIST_SODPP, ent_out_notMNIST_SODPP, MMC_out_notMNIST_SODPP, auroc_out_notMNIST_SODPP, 'MNIST', test='notMNIST', method='SODPP')
print_out_dist_values(acc_out_KMNIST_SODPP, prob_correct_out_KMNIST_SODPP, ent_out_KMNIST_SODPP, MMC_out_KMNIST_SODPP, auroc_out_KMNIST_SODPP, 'MNIST', test='KMNIST', method='SODPP')

### KFAC SODPP

In [ ]:
MNIST_test_in_SODPP_K = predict_second_order_dpp(la_kron, MNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_FMNIST_SODPP_K = predict_second_order_dpp(la_kron, FMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_notMNIST_SODPP_K = predict_second_order_dpp(la_kron, notMNIST_test_loader, timing=True, device=device).cpu().numpy()
MNIST_test_out_KMNIST_SODPP_K = predict_second_order_dpp(la_kron, KMNIST_test_loader, timing=True, device=device).cpu().numpy()

In [ ]:
acc_in_SODPP, prob_correct_in_SODPP, ent_in_SODPP, MMC_in_SODPP = get_in_dist_values(MNIST_test_in_SODPP_K, targets)
acc_out_FMNIST_SODPP, prob_correct_out_FMNIST_SODPP, ent_out_FMNIST_SODPP, MMC_out_FMNIST_SODPP, auroc_out_FMNIST_SODPP = get_out_dist_values(MNIST_test_in_SODPP_K, MNIST_test_out_FMNIST_SODPP_K, targets_FMNIST)
acc_out_notMNIST_SODPP, prob_correct_out_notMNIST_SODPP, ent_out_notMNIST_SODPP, MMC_out_notMNIST_SODPP, auroc_out_notMNIST_SODPP = get_out_dist_values(MNIST_test_in_SODPP_K, MNIST_test_out_notMNIST_SODPP_K, targets_notMNIST)
acc_out_KMNIST_SODPP, prob_correct_out_KMNIST_SODPP, ent_out_KMNIST_SODPP, MMC_out_KMNIST_SODPP, auroc_out_KMNIST_SODPP = get_out_dist_values(MNIST_test_in_SODPP_K, MNIST_test_out_KMNIST_SODPP_K, targets_KMNIST)

In [ ]:
print_in_dist_values(acc_in_SODPP, prob_correct_in_SODPP, ent_in_SODPP, MMC_in_SODPP, 'MNIST', 'SODPP')
print_out_dist_values(acc_out_FMNIST_SODPP, prob_correct_out_FMNIST_SODPP, ent_out_FMNIST_SODPP, MMC_out_FMNIST_SODPP, auroc_out_FMNIST_SODPP, 'MNIST', test='fmnist', method='SODPP')
print_out_dist_values(acc_out_notMNIST_SODPP, prob_correct_out_notMNIST_SODPP, ent_out_notMNIST_SODPP, MMC_out_notMNIST_SODPP, auroc_out_notMNIST_SODPP, 'MNIST', test='notMNIST', method='SODPP')
print_out_dist_values(acc_out_KMNIST_SODPP, prob_correct_out_KMNIST_SODPP, ent_out_KMNIST_SODPP, MMC_out_KMNIST_SODPP, auroc_out_KMNIST_SODPP, 'MNIST', test='KMNIST', method='SODPP')

In [ ]:
break

# Experiments on Rotated MNIST

In [ ]:
import torchvision.transforms as transforms
# rotate 15 degrees
MNIST_transform_r15 = transforms.Compose([
    transforms.RandomRotation(15),
    transforms.ToTensor()
    ])

MNIST_test_r15 = torchvision.datasets.MNIST(
        '~/data/mnist',
        train=False,
        download=False,
        transform=MNIST_transform_r15)

mnist_test_loader_r15 = torch.utils.data.dataloader.DataLoader(
    MNIST_test_r15,
    batch_size=BATCH_SIZE_TEST_MNIST,
    shuffle=False
)

# rotate 30 degrees
MNIST_transform_r30 = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    ])

MNIST_test_r30 = torchvision.datasets.MNIST(
        '~/data/mnist',
        train=False,
        download=False,
        transform=MNIST_transform_r30)

mnist_test_loader_r30 = torch.utils.data.dataloader.DataLoader(
    MNIST_test_r30,
    batch_size=BATCH_SIZE_TEST_MNIST,
    shuffle=False
)

# rotate 45 degrees
MNIST_transform_r45 = transforms.Compose([
    transforms.RandomRotation(45),
    transforms.ToTensor(),
    ])

MNIST_test_r45 = torchvision.datasets.MNIST(
        '~/data/mnist',
        train=False,
        download=False,
        transform=MNIST_transform_r45)

mnist_test_loader_r45 = torch.utils.data.dataloader.DataLoader(
    MNIST_test_r45,
    batch_size=BATCH_SIZE_TEST_MNIST,
    shuffle=False
)

# rotate 60 degrees
MNIST_transform_r60 = transforms.Compose([
    transforms.RandomRotation(60),
    transforms.ToTensor(),
    ])

MNIST_test_r60 = torchvision.datasets.MNIST(
        '~/data/mnist',
        train=False,
        download=False,
        transform=MNIST_transform_r60)

mnist_test_loader_r60 = torch.utils.data.dataloader.DataLoader(
    MNIST_test_r60,
    batch_size=BATCH_SIZE_TEST_MNIST,
    shuffle=False
)

# rotate 75 degrees
MNIST_transform_r75 = transforms.Compose([
    transforms.RandomRotation(75),
    transforms.ToTensor(),
    ])

MNIST_test_r75 = torchvision.datasets.MNIST(
        '~/data/mnist',
        train=False,
        download=False,
        transform=MNIST_transform_r75)

mnist_test_loader_r75 = torch.utils.data.dataloader.DataLoader(
    MNIST_test_r75,
    batch_size=BATCH_SIZE_TEST_MNIST,
    shuffle=False
)

# rotate 90 degrees
MNIST_transform_r90 = transforms.Compose([
    transforms.RandomRotation(90),
    transforms.ToTensor(),
    ])

MNIST_test_r90 = torchvision.datasets.MNIST(
        '~/data/mnist',
        train=False,
        download=False,
        transform=MNIST_transform_r90)

mnist_test_loader_r90 = torch.utils.data.dataloader.DataLoader(
    MNIST_test_r90,
    batch_size=BATCH_SIZE_TEST_MNIST,
    shuffle=False
)

# rotate 105 degrees
MNIST_transform_r105 = transforms.Compose([
    transforms.RandomRotation(105),
    transforms.ToTensor(),
    ])

MNIST_test_r105 = torchvision.datasets.MNIST(
        '~/data/mnist',
        train=False,
        download=False,
        transform=MNIST_transform_r105)

mnist_test_loader_r105 = torch.utils.data.dataloader.DataLoader(
    MNIST_test_r105,
    batch_size=BATCH_SIZE_TEST_MNIST,
    shuffle=False
)

# rotate 120 degrees
MNIST_transform_r120 = transforms.Compose([
    transforms.RandomRotation(120),
    transforms.ToTensor(),
    ])

MNIST_test_r120 = torchvision.datasets.MNIST(
        '~/data/mnist',
        train=False,
        download=False,
        transform=MNIST_transform_r120)

mnist_test_loader_r120 = torch.utils.data.dataloader.DataLoader(
    MNIST_test_r120,
    batch_size=BATCH_SIZE_TEST_MNIST,
    shuffle=False
)

# rotate 135 degrees
MNIST_transform_r135 = transforms.Compose([
    transforms.RandomRotation(135),
    transforms.ToTensor(),
    ])

MNIST_test_r135 = torchvision.datasets.MNIST(
        '~/data/mnist',
        train=False,
        download=False,
        transform=MNIST_transform_r135)

mnist_test_loader_r135 = torch.utils.data.dataloader.DataLoader(
    MNIST_test_r135,
    batch_size=BATCH_SIZE_TEST_MNIST,
    shuffle=False
)

# rotate 150 degrees
MNIST_transform_r150= transforms.Compose([
    transforms.RandomRotation(150),
    transforms.ToTensor(),
    ])

MNIST_test_r150 = torchvision.datasets.MNIST(
        '~/data/mnist',
        train=False,
        download=False,
        transform=MNIST_transform_r150)

mnist_test_loader_r150 = torch.utils.data.dataloader.DataLoader(
    MNIST_test_r150,
    batch_size=BATCH_SIZE_TEST_MNIST,
    shuffle=False
)

# rotate 165 degrees
MNIST_transform_r165 = transforms.Compose([
    transforms.RandomRotation(165),
    transforms.ToTensor(),
    ])

MNIST_test_r165 = torchvision.datasets.MNIST(
        '~/data/mnist',
        train=False,
        download=False,
        transform=MNIST_transform_r165)

mnist_test_loader_r165 = torch.utils.data.dataloader.DataLoader(
    MNIST_test_r165,
    batch_size=BATCH_SIZE_TEST_MNIST,
    shuffle=False
)

# rotate 180 degrees
MNIST_transform_r180 = transforms.Compose([
    transforms.RandomRotation(180),
    transforms.ToTensor(),
    ])

MNIST_test_r180 = torchvision.datasets.MNIST(
        '~/data/mnist',
        train=False,
        download=False,
        transform=MNIST_transform_r180)

mnist_test_loader_r180 = torch.utils.data.dataloader.DataLoader(
    MNIST_test_r180,
    batch_size=BATCH_SIZE_TEST_MNIST,
    shuffle=False
)

In [ ]:
from sklearn.metrics import brier_score_loss

## helper function: given a dataloader compute accuracy and brier score
def get_acc_brier(dataloader, targets, num_samples=1000):
    
    # compute sampling results
    mnist_rotated_D = predict_diagonal_sampling(mnist_model, dataloader, M_W_post_D, M_b_post_D, C_W_post_D, C_b_post_D, verbose=False, cuda=True, timing=False, n_samples=num_samples).cpu().numpy()
    
    # compute LB results
    mnist_rotated_LB = predict_LB(mnist_model, dataloader, M_W_post_D, M_b_post_D, C_W_post_D, C_b_post_D, verbose=False, cuda=True, timing=False).cpu().numpy()
    mnist_rotated_LBn = mnist_rotated_LB/mnist_rotated_LB.sum(1).reshape(-1,1)
    
    # accuracy for sampling and LB
    acc_D = np.mean(np.argmax(mnist_rotated_D, 1) == targets)
    acc_LB = np.mean(np.argmax(mnist_rotated_LBn, 1) == targets)
    
    # get brier score for sampling and LB
    pred_at_true_D = np.array([mnist_rotated_D[i, j] for i, j in enumerate(targets)])
    pred_at_true_LBn = np.array([mnist_rotated_LBn[i, j] for i, j in enumerate(targets)])
    
    brier_D = brier_score_loss(np.ones_like(pred_at_true_D), pred_at_true_D)
    brier_LB = brier_score_loss(np.ones_like(pred_at_true_LBn), pred_at_true_LBn)
    
    return(acc_D, acc_LB, brier_D, brier_LB)

In [ ]:
# just a test
get_acc_brier(mnist_test_loader_r15, targets)

In [ ]:
## predict on all distributions and compute accuracy and brier score

dataloader_list = [mnist_test_loader, mnist_test_loader_r15, mnist_test_loader_r30, mnist_test_loader_r45,
                  mnist_test_loader_r60, mnist_test_loader_r75, mnist_test_loader_r90, mnist_test_loader_r105,
                  mnist_test_loader_r120, mnist_test_loader_r135, mnist_test_loader_r150, mnist_test_loader_r165, 
                  mnist_test_loader_r180]

Acc_D_list = []
Acc_LB_list = []
Brier_D_list = []
Brier_LB_list = []

for i, loader_ in enumerate(dataloader_list):
    print(i)
    Acc_D_, Acc_LB_, Brier_D_, Brier_LB_ = get_acc_brier(loader_, targets)
    Acc_D_list.append(Acc_D_)
    Acc_LB_list.append(Acc_LB_)
    Brier_D_list.append(Brier_D_)
    Brier_LB_list.append(Brier_LB_)
    

In [ ]:
# Make inline plots vector graphics
import matplotlib
from IPython.display import set_matplotlib_formats

set_matplotlib_formats("pdf", "svg")

matplotlib.rc("font", **{"family": "serif", "serif": ["Computer Modern"]})
plt.rcParams["text.usetex"] = True
plt.rcParams["text.latex.preamble"] = r"\usepackage{amsfonts} \usepackage{amsmath}"

In [ ]:
print(Acc_D_list)
print(Acc_LB_list)
print(Brier_D_list)
print(Brier_LB_list)

In [ ]:
#### compare over 5 seeds: 123, 124, 125, 126, 127
Acc_D_all = np.array([
    [0.9906, 0.981, 0.9401, 0.8593, 0.7388, 0.6375, 0.5627, 0.5049, 0.4703, 0.4533, 0.4363, 0.4287, 0.4268],
    [0.9889, 0.9814, 0.9433, 0.8518, 0.7337, 0.6403, 0.561, 0.5016, 0.4669, 0.4306, 0.4292, 0.4104, 0.4183],
    [0.9904, 0.9829, 0.946, 0.8582, 0.747, 0.6495, 0.5602, 0.5075, 0.4684, 0.4437, 0.4334, 0.4245, 0.4217],
    [0.9875, 0.9776, 0.9353, 0.8422, 0.7212, 0.6377, 0.55, 0.4896, 0.4497, 0.4328, 0.416, 0.4117, 0.4095],
    [0.9894, 0.9816, 0.9451, 0.8522, 0.7403, 0.6391, 0.5565, 0.4994, 0.4656, 0.4308, 0.4284, 0.4221, 0.4152]
])

Acc_LB_all = np.array([
    [0.9907, 0.983, 0.9394, 0.8526, 0.7419, 0.6386, 0.5703, 0.5152, 0.4702, 0.4484, 0.4335, 0.4236, 0.4182],
    [0.9889, 0.9804, 0.9471, 0.8547, 0.7411, 0.6296, 0.5624, 0.4938, 0.4609, 0.4404, 0.4295, 0.4172, 0.4152],
    [0.9905, 0.9831, 0.9436, 0.8616, 0.7461, 0.6517, 0.5647, 0.5127, 0.4673, 0.4454, 0.4273, 0.4291, 0.415],
    [0.9876, 0.9799, 0.9398, 0.851, 0.7352, 0.6368, 0.5445, 0.4992, 0.4632, 0.4269, 0.42, 0.417, 0.4181],
    [0.9893, 0.9808, 0.939, 0.8465, 0.7338, 0.6312, 0.5573, 0.505, 0.4539, 0.4458, 0.4158, 0.4189, 0.4145]
])

Brier_D_all = np.array([
    [0.008377700125184932, 0.016529366393931315, 0.05263197122933036, 0.12622978433992207, 0.23428155265648049, 0.33001102154197504, 0.40364649390174306, 0.45833621095510935, 0.48925685879783176, 0.5084873524892697, 0.5265005890140535, 0.5351897862160594, 0.535655688821819],
    [0.010744372283432188, 0.01861786528128727, 0.04921392785665237, 0.130553247122029, 0.23708335123859933, 0.3264322724129021, 0.4020700676185214, 0.45858313992431765, 0.49138653020376494, 0.5267105075863001, 0.5302083038117827, 0.5471840382483096, 0.5406172102829678],
    [0.008842406659404037, 0.01589756693721347, 0.04813633596723046, 0.12450638545589118, 0.2251983306557004, 0.31596819055291797, 0.39958471677594976, 0.4511044612864316, 0.4908296869743894, 0.513979687459884, 0.5228012975599322, 0.5331324584577392, 0.5366587666968312],
    [0.010730850229531219, 0.019535984307980356, 0.054532256239393395, 0.13633893796280153, 0.24614911889753252, 0.32741910714203704, 0.4095507809925442, 0.4699407662727923, 0.5086184210935191, 0.525650285971151, 0.544528220493102, 0.5479489062984142, 0.5516986011718078],
    [0.009795412763015064, 0.01817015340220003, 0.04894315094823046, 0.12908701202114847, 0.23282846945381916, 0.3285278649762006, 0.4081318056690933, 0.46182442301821763, 0.4973149243671581, 0.5269344762781906, 0.5322562363481955, 0.5380640556117866, 0.5471252220414157]
])

Brier_LB_all = np.array([
    [0.007844242911062461, 0.015166313465830961, 0.05119071505966292, 0.13220723722716274, 0.23246172960926015, 0.3326357106190896, 0.4017439850542811, 0.45393964088759164, 0.49749242841385766, 0.5211752780472316, 0.5353101111950346, 0.5431750248692432, 0.5509519585301749],
    [0.009505463873917114, 0.016971207786010672, 0.0470778754224552, 0.12817579122361247, 0.2349776937700805, 0.339070595213455, 0.40568261118347, 0.4740733457035303, 0.5030623144832805, 0.5253332179429556, 0.5360191577762338, 0.549535112754868, 0.5511099660875561],
    [0.0076984171595750415, 0.014756499162356471, 0.0469422462627717, 0.12101585517824154, 0.22767872259093588, 0.31762503643888157, 0.4011728288118217, 0.4516875142101383, 0.4954514924412558, 0.5178717196068767, 0.5371807295796656, 0.5349908080424384, 0.5497019253514567],
    [0.01001831719153492, 0.016729483129191477, 0.05172484471829214, 0.13075484635707604, 0.23891760292945455, 0.3334325462508124, 0.42297503706889894, 0.4695227387634461, 0.5033610244309235, 0.5387111053297119, 0.5473753173717428, 0.5506346709903308, 0.5503556560492305],
    [0.008755419081030589, 0.01618864623953837, 0.05156303842716825, 0.13417919609729662, 0.23963138466766964, 0.34131713744443276, 0.4110757904444741, 0.46371903749768173, 0.5107108175950072, 0.5222953217647085, 0.5486314035009504, 0.5470332878374046, 0.5531479907833964]   
])

In [ ]:
Acc_D_all_mean = np.mean(Acc_D_all, axis=0)
Acc_D_all_std = np.std(Acc_D_all, axis=0)

Acc_LB_all_mean = np.mean(Acc_LB_all, axis=0)
Acc_LB_all_std = np.std(Acc_LB_all, axis=0)

Brier_D_all_mean = np.mean(Brier_D_all, axis=0)
Brier_D_all_std = np.std(Brier_D_all, axis=0)

Brier_LB_all_mean = np.mean(Brier_LB_all, axis=0)
Brier_LB_all_std = np.std(Brier_LB_all, axis=0)

In [ ]:
# make a figure for accuracy
x_labels = ['test', '15$^{\circ}$', '30$^{\circ}$', '45$^{\circ}$', '60$^{\circ}$', '75$^{\circ}$', '90$^{\circ}$',
            '105$^{\circ}$', '120$^{\circ}$', '135$^{\circ}$', '150$^{\circ}$', '165$^{\circ}$', '180$^{\circ}$']

plt.figure(figsize=(5, 1.5))
plt.plot(x_labels, Acc_D_all_mean, color='blue', label='MCMC')
plt.plot(x_labels, Acc_LB_all_mean, color='firebrick', label='LB')
plt.ylim(0,1)
plt.ylabel('Accuracy', size=15)
#plt.legend()
plt.xticks(x_labels, x_labels, rotation='30', size=13)

plt.tight_layout()

plt.savefig('figures/LB_vs_MCMC_Acc.pdf')
plt.show();

In [ ]:
# make a figure for the brier score

plt.figure(figsize=(5, 1.5))
plt.plot(x_labels, Brier_D_list, color='blue', label='MCMC')
plt.plot(x_labels, Brier_LB_list, color='firebrick', label='LB')
plt.ylim(0,0.8)
plt.ylabel('Brier', size=15)
plt.legend()
plt.xticks(x_labels, x_labels, rotation='30', size=13)

plt.tight_layout()

plt.savefig('figures/LB_vs_MCMC_Brier.pdf')
plt.show();

In [ ]:
# make a subplot

fig, ax = plt.subplots(2, 1, figsize=(5, 3))

ax[0].plot(x_labels, Acc_D_all_mean, color='blue', label='MCMC')
ax[0].plot(x_labels, Acc_LB_all_mean, color='firebrick', label='LB')
ax[0].set_ylim(0,1)
ax[0].set_ylabel('Accuracy', size=15)
ax[0].set_xticklabels([])

ax[1].plot(x_labels, Brier_D_list, color='blue', label='MCMC')
ax[1].plot(x_labels, Brier_LB_list, color='firebrick', label='LB')
ax[1].set_ylim(0,0.8)
ax[1].set_ylabel('Brier', size=15)
ax[1].legend()
ax[1].set_xticklabels(x_labels, rotation=30, size=13)


plt.tight_layout()

plt.savefig('figures/LB_vs_MCMC_Acc_Brier.pdf')
plt.show();